In [1]:
import pandas as pd
import datetime as dt
import geopandas as gp
import urllib2
import matplotlib.pyplot as plt
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
# import datetime as dt
import re
from shapely.geometry import Point
%matplotlib inline

In [2]:
MTA_TURNSTILE_URL = "http://web.mta.info/developers/turnstile.html"
MTA_FILE_ROOT_URL = "http://web.mta.info/developers/"

In [3]:
# '''
# slightly modified from https://github.com/piratefsh/mta-turnstile-scraper/blob/master/mta_scraper.py'''

def get_site():
    f = urllib2.urlopen(MTA_TURNSTILE_URL)
    content = f.read()
    return content

def get_turnstile_links(html):
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all('a')
    turnstile_links = [(link.text, MTA_FILE_ROOT_URL + link['href']) for link in links if re.match('.*day.*20..', link.text)]
    return turnstile_links 

def get_links_by_date(start, end):
    links = get_links()
    in_range = []
    for text,link in links:
        date = datetime.strptime(text , "%A, %B %d, %Y") 
        if date >= start and date <= end:
            in_range.append((text, link))
    
    return in_range

def get_links():
    return get_turnstile_links(get_site())

In [4]:
metadata = 'http://web.mta.info/developers/resources/nyct/turnstile/ts_Field_Description_pre-10-18-2014.txt'
f = urllib2.urlopen(metadata)
content = f.readlines()
col_names_14 = str(content[2]).split(',')

def turnstile_df(links, post2014 = True): 
    '''takes links and returns dataframe with all data in one
    pre 2014 was captured differently and thus requires a second argument for correct columns'''
    df = pd.DataFrame()
    if post2014 == True: 
        for text, link in links: 
            temp_df = pd.read_csv(link)
            temp_df['DATE'] = temp_df['DATE'] + ' ' + temp_df['TIME']
            print "completed " + link
#             temp_df['DATE'] = pd.to_datetime(temp_df['DATE'])
            df = df.append(temp_df, ignore_index = True)
    elif post2014 == False: 
        for text, link in links: 
            temp_df = pd.read_csv(link, header = None, names = col_names_14, parse_dates = True,
                                 infer_datetime_format = True)
            df = df.append(temp_df, ignore_index = True)
    return df

In [7]:
a = get_site()
b = get_turnstile_links(a)
start = datetime(2013, 01, 05)
end = datetime(2014, 10, 11)
desired = get_links_by_date(start, end)
y1314 = turnstile_df(desired, post2014= False)

In [11]:
y1314.to_csv("../../Data/subway/subways_1314.csv")

In [14]:
data = pd.read_csv("../../Data/subway/subways_1314.csv")

In [16]:
data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [25]:
data

,C/A,UNIT,SCP,DATE1,TIME1,DESC1,ENTRIES1,EXITS1,DATE2,TIME2,...,DATE7,TIME7,DESC7,ENTRIES7,EXITS7,DATE8,TIME8,DESC8,ENTRIES8,EXITS8
0,A002,R051,02-00-00,10-04-14,00:00:00,REGULAR,4810694,1632419,10-04-14,04:00:00,...,10-05-14,00:00:00,REGULAR,4811491.0,1632626.0,10-05-14,04:00:00,REGULAR,4811518.0,1632637.0
1,A002,R051,02-00-00,10-05-14,08:00:00,REGULAR,4811532,1632653,10-05-14,12:00:00,...,10-06-14,08:00:00,REGULAR,4812318.0,1632950.0,10-06-14,12:00:00,REGULAR,4812502.0,1633231.0
2,A002,R051,02-00-00,10-06-14,16:00:00,REGULAR,4812783,1633296,10-06-14,20:00:00,...,10-07-14,16:00:00,REGULAR,4814492.0,1633908.0,10-07-14,20:00:00,REGULAR,4815446.0,1633967.0
3,A002,R051,02-00-00,10-08-14,00:00:00,REGULAR,4815662,1633996,10-08-14,04:00:00,...,10-09-14,00:00:00,REGULAR,4817425.0,1634564.0,10-09-14,04:00:00,REGULAR,4817459.0,1634574.0
4,A002,R051,02-00-00,10-09-14,08:00:00,REGULAR,4817510,1634680,10-09-14,12:00:00,...,10-10-14,07:30:04,REGULAR,4819241.0,1635244.0,10-10-14,08:00:00,REGULAR,4819256.0,1635283.0
5,A002,R051,02-00-00,10-10-14,12:00:00,REGULAR,4819429,1635564,10-10-14,16:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,A002,R051,02-00-01,10-04-14,00:00:00,REGULAR,4480547,976834,10-04-14,04:00:00,...,10-05-14,00:00:00,REGULAR,4481231.0,976999.0,10-05-14,04:00:00,REGULAR,4481255.0,977001.0
7,A002,R051,02-00-01,10-05-14,08:00:00,REGULAR,4481268,977011,10-05-14,12:00:00,...,10-06-14,08:00:00,REGULAR,4481899.0,977203.0,10-06-14,12:00:00,REGULAR,4482065.0,977349.0
8,A002,R051,02-00-01,10-06-14,16:00:00,REGULAR,4482308,977389,10-06-14,20:00:00,...,10-07-14,16:00:00,REGULAR,4483651.0,977722.0,10-07-14,20:00:00,REGULAR,4484439.0,977773.0
9,A002,R051,02-00-01,10-08-14,00:00:00,REGULAR,4484604,977789,10-08-14,04:00:00,...,10-09-14,00:00:00,REGULAR,4486022.0,978122.0,10-09-14,04:00:00,REGULAR,4486048.0,978126.0


In [20]:
dtest = data.iloc[:10]

In [22]:
df = pd.DataFrame(columns=['C/A', 'UNIT', 'SCP', 'DATE', 'TIME', 'DESC', 'ENTRIES', 'EXITS'])

In [ ]:
import csv

def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row
            
for row in csvRows('../../Data/subway/subways_1314.csv'):
    
        

In [19]:
data.iloc[0]

C/A                A002
UNIT               R051
SCP            02-00-00
DATE1          10-04-14
TIME1          00:00:00
DESC1           REGULAR
ENTRIES1        4810694
EXITS1          1632419
DATE2          10-04-14
TIME2          04:00:00
DESC2           REGULAR
ENTRIES2    4.81073e+06
EXITS2      1.63242e+06
DATE3          10-04-14
TIME3          08:00:00
DESC3        RECOVR AUD
ENTRIES3    4.81075e+06
EXITS3      1.63245e+06
DATE4          10-04-14
TIME4          12:00:00
DESC4           REGULAR
ENTRIES4    4.81083e+06
EXITS4      1.63253e+06
DATE5          10-04-14
TIME5          16:00:00
DESC5           REGULAR
ENTRIES5    4.81103e+06
EXITS5      1.63258e+06
DATE6          10-04-14
TIME6          20:00:00
DESC6           REGULAR
ENTRIES6     4.8113e+06
EXITS6      1.63261e+06
DATE7          10-05-14
TIME7          00:00:00
DESC7           REGULAR
ENTRIES7    4.81149e+06
EXITS7      1.63263e+06
DATE8          10-05-14
TIME8          04:00:00
DESC8           REGULAR
ENTRIES8    4.81

In [25]:
data = data[(data.DIVISION != 'PTH') & (data.DIVISION != 'SRT') & (data.DIVISION != 'RIT')]

In [27]:
data.drop(['Unnamed: 0'],
        inplace = True, axis = 1)

In [28]:
data.rename(columns={'EXITS                                                               ': 'EXITS'}, inplace=True)

In [29]:
data.loc[:,'uni'] = data.loc[:,'STATION'] + " " + data.loc[:,'LINENAME'] + " " + data.loc[:,'DIVISION']

In [30]:
data.columns

Index([u'C/A', u'UNIT', u'SCP', u'STATION', u'LINENAME', u'DIVISION', u'DATE',
       u'TIME', u'DESC', u'ENTRIES', u'EXITS', u'uni'],
      dtype='object')

In [31]:
df = data[data.DESC == 'REGULAR']
df = df.groupby([u'C/A', u'UNIT', u'uni', u'SCP', u'DATE']).sum()

In [32]:
df["ENTRIES_DIFF"] = df.groupby(level=[0,1,2,3])['ENTRIES'].transform(lambda x: abs(x - x.shift()))
df["EXITS_DIFF"] = df.groupby(level=[0,1,2,3])['EXITS'].transform(lambda x: abs(x - x.shift()))

In [33]:
df.dropna(inplace=True)

In [34]:
df.reset_index(inplace=True)

In [35]:
df.ENTRIES_DIFF.quantile(0.9999)

20100.60959997773

In [36]:
reduced = df[df.ENTRIES_DIFF < df.ENTRIES_DIFF.quantile(0.9999)]['ENTRIES_DIFF']
    
lower_thresh = 0
upper_thresh = reduced.mean() + 3 * reduced.std()
                                                                           
outlier_mask = df['ENTRIES_DIFF'].map(lambda x: lower_thresh <= x <= upper_thresh)
    
df = df[outlier_mask]

In [37]:
df = df.set_index('DATE').groupby([pd.TimeGrouper('D'), 'uni']).sum()

In [38]:
df.reset_index(['uni'], inplace=True)

In [39]:
df.drop(['ENTRIES', 'EXITS'], axis=1, inplace=True)

In [40]:
df.reset_index(inplace=True)

In [41]:
df.columns

Index([u'DATE', u'uni', u'ENTRIES_DIFF', u'EXITS_DIFF'], dtype='object')

In [42]:
stations = df['uni']

In [20]:
stations.to_csv("../../Data/subway/stations.csv")

In [21]:
# df.DATE.describe()

# Subways

In [43]:
subways = pd.read_csv('http://web.mta.info/developers/data/nyct/subway/StationEntrances.csv',
                     usecols=['Station_Name', 'Station_Latitude','Station_Longitude', 'Division','Route_1', 'Route_2', 
                              'Route_3', 'Route_4', 'Route_5', 'Route_6', 'Route_7', 'Route_8', 'Route_9', 'Route_10',
                              'Route_11'])

In [44]:
for i in range(len(subways)):
    
    if ((str(subways.loc[i,'Route_2']) != 'nan') & (str(subways.loc[i,'Route_3']) != 'nan') & 
        (str(subways.loc[i,'Route_4']) != 'nan') & (str(subways.loc[i,'Route_5']) != 'nan') &
        (str(subways.loc[i,'Route_6']) != 'nan') & (str(subways.loc[i,'Route_7']) != 'nan') &
        (str(subways.loc[i,'Route_8']) != 'nan') & (str(subways.loc[i,'Route_9']) != 'nan') &
        (str(subways.loc[i,'Route_10']) != 'nan') & (str(subways.loc[i,'Route_11']) != 'nan')):
        subways.loc[i,'LINENAME'] = str(str(subways.loc[i,'Route_1']) + str(subways.loc[i,'Route_2']) + 
                                        str(subways.loc[i,'Route_3']) + str(subways.loc[i,'Route_4']) + 
                                        str(subways.loc[i,'Route_5']) + str(subways.loc[i,'Route_6']) + 
                                        str(subways.loc[i,'Route_7']) + str(subways.loc[i,'Route_8']) + 
                                        str(subways.loc[i,'Route_9']) + str(subways.loc[i,'Route_10']) + 
                                        str(subways.loc[i,'Route_11']) )
        
    elif ((str(subways.loc[i,'Route_2']) != 'nan') & (str(subways.loc[i,'Route_3']) != 'nan') & 
          (str(subways.loc[i,'Route_4']) != 'nan') & (str(subways.loc[i,'Route_5']) != 'nan') &
          (str(subways.loc[i,'Route_6']) != 'nan') & (str(subways.loc[i,'Route_7']) != 'nan') &
          (str(subways.loc[i,'Route_8']) != 'nan') & (str(subways.loc[i,'Route_9']) != 'nan') &
          (str(subways.loc[i,'Route_10']) != 'nan') &(str(subways.loc[i,'Route_11']) == 'nan')):
        subways.loc[i,'LINENAME'] = str(str(subways.loc[i,'Route_1']) + str(subways.loc[i,'Route_2']) + 
                                        str(subways.loc[i,'Route_3']) + str(subways.loc[i,'Route_4']) + 
                                        str(subways.loc[i,'Route_5']) + str(subways.loc[i,'Route_6']) + 
                                        str(subways.loc[i,'Route_7']) + str(subways.loc[i,'Route_8']) + 
                                        str(subways.loc[i,'Route_9']) + str(subways.loc[i,'Route_10']))

    elif ((str(subways.loc[i,'Route_2']) != 'nan') & (str(subways.loc[i,'Route_3']) != 'nan') & 
          (str(subways.loc[i,'Route_4']) != 'nan') & (str(subways.loc[i,'Route_5']) != 'nan') &
          (str(subways.loc[i,'Route_6']) != 'nan') & (str(subways.loc[i,'Route_7']) != 'nan') &
          (str(subways.loc[i,'Route_8']) != 'nan') & (str(subways.loc[i,'Route_9']) != 'nan') &
          (str(subways.loc[i,'Route_10']) == 'nan') & (str(subways.loc[i,'Route_11']) == 'nan')):
        subways.loc[i,'LINENAME'] = str(str(subways.loc[i,'Route_1']) + str(subways.loc[i,'Route_2']) + 
                                        str(subways.loc[i,'Route_3']) + str(subways.loc[i,'Route_4']) + 
                                        str(subways.loc[i,'Route_5']) + str(subways.loc[i,'Route_6']) + 
                                        str(subways.loc[i,'Route_7']) + str(subways.loc[i,'Route_8']) + 
                                        str(subways.loc[i,'Route_9']) )
    
    elif ((str(subways.loc[i,'Route_2']) != 'nan') & (str(subways.loc[i,'Route_3']) != 'nan') &
          (str(subways.loc[i,'Route_4']) != 'nan') & (str(subways.loc[i,'Route_5']) != 'nan') &
          (str(subways.loc[i,'Route_6']) != 'nan') & (str(subways.loc[i,'Route_7']) != 'nan') &
          (str(subways.loc[i,'Route_8']) != 'nan') & (str(subways.loc[i,'Route_9']) == 'nan') &
          (str(subways.loc[i,'Route_10']) == 'nan') & (str(subways.loc[i,'Route_11']) == 'nan')):
        subways.loc[i,'LINENAME'] = str(str(subways.loc[i,'Route_1']) + str(subways.loc[i,'Route_2']) + 
                                        str(subways.loc[i,'Route_3']) + str(subways.loc[i,'Route_4']) + 
                                        str(subways.loc[i,'Route_5']) + str(subways.loc[i,'Route_6']) + 
                                        str(subways.loc[i,'Route_7']) + str(subways.loc[i,'Route_8']) )

    elif ((str(subways.loc[i,'Route_2']) != 'nan') & (str(subways.loc[i,'Route_3']) != 'nan') &
          (str(subways.loc[i,'Route_4']) != 'nan') & (str(subways.loc[i,'Route_5']) != 'nan') &
          (str(subways.loc[i,'Route_6']) != 'nan') & (str(subways.loc[i,'Route_7']) != 'nan') &
          (str(subways.loc[i,'Route_8']) == 'nan') & (str(subways.loc[i,'Route_9']) == 'nan') &
          (str(subways.loc[i,'Route_10']) == 'nan') & (str(subways.loc[i,'Route_11']) == 'nan')):
        subways.loc[i,'LINENAME'] = str(str(subways.loc[i,'Route_1']) + str(subways.loc[i,'Route_2']) + 
                                        str(subways.loc[i,'Route_3']) + str(subways.loc[i,'Route_4']) + 
                                        str(subways.loc[i,'Route_5']) + str(subways.loc[i,'Route_6']) + 
                                        str(subways.loc[i,'Route_7']))    

    elif ((str(subways.loc[i,'Route_2']) != 'nan') & (str(subways.loc[i,'Route_3']) != 'nan') &
          (str(subways.loc[i,'Route_4']) != 'nan') & (str(subways.loc[i,'Route_5']) != 'nan') &
          (str(subways.loc[i,'Route_6']) != 'nan') & (str(subways.loc[i,'Route_7']) == 'nan') &
          (str(subways.loc[i,'Route_8']) == 'nan') & (str(subways.loc[i,'Route_9']) == 'nan') &
          (str(subways.loc[i,'Route_10']) == 'nan') & (str(subways.loc[i,'Route_11']) == 'nan')):
        subways.loc[i,'LINENAME'] = str(str(subways.loc[i,'Route_1']) + str(subways.loc[i,'Route_2']) + 
                                        str(subways.loc[i,'Route_3']) + str(subways.loc[i,'Route_4']) + 
                                        str(subways.loc[i,'Route_5']) + str(subways.loc[i,'Route_6']) )

    elif ((str(subways.loc[i,'Route_2']) != 'nan') & (str(subways.loc[i,'Route_3']) != 'nan') &
          (str(subways.loc[i,'Route_4']) != 'nan') & (str(subways.loc[i,'Route_5']) != 'nan') &
          (str(subways.loc[i,'Route_6']) == 'nan') & (str(subways.loc[i,'Route_7']) == 'nan') &
          (str(subways.loc[i,'Route_8']) == 'nan') & (str(subways.loc[i,'Route_9']) == 'nan') &
          (str(subways.loc[i,'Route_10']) == 'nan') & (str(subways.loc[i,'Route_11']) == 'nan')):
        subways.loc[i,'LINENAME'] = str(str(subways.loc[i,'Route_1']) + str(subways.loc[i,'Route_2']) + 
                                        str(subways.loc[i,'Route_3']) + str(subways.loc[i,'Route_4']) + 
                                        str(subways.loc[i,'Route_5']) )
        
    elif ((str(subways.loc[i,'Route_2']) != 'nan') & (str(subways.loc[i,'Route_3']) != 'nan') &
          (str(subways.loc[i,'Route_4']) != 'nan') & (str(subways.loc[i,'Route_5']) == 'nan') &
          (str(subways.loc[i,'Route_6']) == 'nan') & (str(subways.loc[i,'Route_7']) == 'nan') &
          (str(subways.loc[i,'Route_8']) == 'nan') & (str(subways.loc[i,'Route_9']) == 'nan') &
          (str(subways.loc[i,'Route_10']) == 'nan') & (str(subways.loc[i,'Route_11']) == 'nan')):
        subways.loc[i,'LINENAME'] = str(str(subways.loc[i,'Route_1']) + str(subways.loc[i,'Route_2']) + 
                                        str(subways.loc[i,'Route_3']) + 
                                        str(subways.loc[i,'Route_4']) )

    elif ((str(subways.loc[i,'Route_2']) != 'nan') & (str(subways.loc[i,'Route_3']) != 'nan') &
          (str(subways.loc[i,'Route_4']) == 'nan') & (str(subways.loc[i,'Route_5']) == 'nan') &
          (str(subways.loc[i,'Route_6']) == 'nan') & (str(subways.loc[i,'Route_7']) == 'nan') &
          (str(subways.loc[i,'Route_8']) == 'nan') & (str(subways.loc[i,'Route_9']) == 'nan') &
          (str(subways.loc[i,'Route_10']) == 'nan') & (str(subways.loc[i,'Route_11']) == 'nan')):
        subways.loc[i,'LINENAME'] = str(str(subways.loc[i,'Route_1']) + str(subways.loc[i,'Route_2']) + 
                                        str(subways.loc[i,'Route_3']) )
        
    elif ((str(subways.loc[i,'Route_2']) != 'nan') & (str(subways.loc[i,'Route_3']) == 'nan') &
          (str(subways.loc[i,'Route_4']) == 'nan') & (str(subways.loc[i,'Route_5']) == 'nan') &
          (str(subways.loc[i,'Route_6']) == 'nan') & (str(subways.loc[i,'Route_7']) == 'nan') &
          (str(subways.loc[i,'Route_8']) == 'nan') & (str(subways.loc[i,'Route_9']) == 'nan') &
          (str(subways.loc[i,'Route_10']) == 'nan') & (str(subways.loc[i,'Route_11']) == 'nan')):
        subways.loc[i,'LINENAME'] = str(str(subways.loc[i,'Route_1']) + str(subways.loc[i,'Route_2']) )

    elif ((str(subways.loc[i,'Route_2']) == 'nan') & (str(subways.loc[i,'Route_3']) == 'nan') &
          (str(subways.loc[i,'Route_4']) == 'nan') & (str(subways.loc[i,'Route_5']) == 'nan') &
          (str(subways.loc[i,'Route_6']) == 'nan') & (str(subways.loc[i,'Route_7']) == 'nan') &
          (str(subways.loc[i,'Route_8']) == 'nan') & (str(subways.loc[i,'Route_9']) == 'nan') &
          (str(subways.loc[i,'Route_10']) == 'nan') & (str(subways.loc[i,'Route_11']) == 'nan')):
        subways.loc[i,'LINENAME'] = str(str(subways.loc[i,'Route_1']))

In [45]:
subways.loc[:,'uni'] = subways.loc[:,'Station_Name'] + " " + subways.loc[:,'LINENAME'] + " " + subways.loc[:,'Division']

In [46]:
subways.drop(['Route_1','Route_2','Route_3','Route_4','Route_5','Route_6','Route_7', 'Route_8', 'Route_9', 'Route_10',
              'Route_11'],inplace=True,axis=1)
subways.reset_index(drop=True, inplace=True)

In [47]:
subways.drop_duplicates(inplace = True)
subways.reset_index(drop=True, inplace=True)

In [48]:
subways.head()

,Division,Station_Name,Station_Latitude,Station_Longitude,LINENAME,uni
0,BMT,Ditmars Blvd,40.775036,-73.912034,NQ,Ditmars Blvd NQ BMT
1,BMT,Astoria Blvd-Hoyt Av,40.770258,-73.917843,NQ,Astoria Blvd-Hoyt Av NQ BMT
2,BMT,30 Av-Grand Av,40.766779,-73.921479,NQ,30 Av-Grand Av NQ BMT
3,BMT,Broadway,40.761820,-73.925508,NQ,Broadway NQ BMT
4,BMT,36 Av-Washington Av,40.756804,-73.929575,NQ,36 Av-Washington Av NQ BMT


In [28]:
subways.to_csv("../Data/subway/subway_locations.csv")

In [29]:
len(subways)

474

# Subway Mapper

In [50]:
mapper = pd.read_csv("../../Data/subway/stations_mapper.csv")

In [51]:
df.head(3)

,DATE,uni,ENTRIES_DIFF,EXITS_DIFF
0,2014-10-11,1 AVE L BMT,13150.0,15662.0
1,2014-10-11,103 ST 1 IRT,9486.0,3681.0
2,2014-10-11,103 ST 6 IRT,8584.0,7402.0


In [52]:
subways.head(3)

,Division,Station_Name,Station_Latitude,Station_Longitude,LINENAME,uni
0,BMT,Ditmars Blvd,40.775036,-73.912034,NQ,Ditmars Blvd NQ BMT
1,BMT,Astoria Blvd-Hoyt Av,40.770258,-73.917843,NQ,Astoria Blvd-Hoyt Av NQ BMT
2,BMT,30 Av-Grand Av,40.766779,-73.921479,NQ,30 Av-Grand Av NQ BMT


In [53]:
mapper.head(3)

,stations,Map
0,103 ST 1 IRT,103rd St 1 IRT
1,103 ST 6 IRT,103rd St 6 IRT
2,103 ST-CORONA 7 IRT,103rd St 7 IRT


In [81]:
mapper.reset_index(inplace= True)

In [82]:
mapper.head()

,index,stations,Map
0,0,103 ST 1 IRT,103rd St 1 IRT
1,1,103 ST 6 IRT,103rd St 6 IRT
2,2,103 ST-CORONA 7 IRT,103rd St 7 IRT
3,3,103 ST BC IND,103rd St BC IND
4,4,104 ST JZ BMT,104th St-102nd St JZ BMT


In [83]:
mapper.columns = ['StationID', 'stations','Map']

In [84]:
len(mapper.stations.unique())

716

In [98]:
df_merged = pd.merge(df, mapper, left_on='uni', right_on='stations', how='left')

In [99]:
df_merged.columns

Index([u'DATE', u'uni', u'ENTRIES_DIFF', u'EXITS_DIFF', u'StationID',
       u'stations', u'Map'],
      dtype='object')

In [100]:
df_merged.dropna(inplace=True)

In [101]:
df_merged.head()

,DATE,uni,ENTRIES_DIFF,EXITS_DIFF,StationID,stations,Map
0,2014-10-11,1 AVE L BMT,13150.0,15662.0,75.0,1 AVE L BMT,1st Av L BMT
1,2014-10-11,103 ST 1 IRT,9486.0,3681.0,0.0,103 ST 1 IRT,103rd St 1 IRT
2,2014-10-11,103 ST 6 IRT,8584.0,7402.0,1.0,103 ST 6 IRT,103rd St 6 IRT
3,2014-10-11,103 ST BC IND,2266.0,2193.0,3.0,103 ST BC IND,103rd St BC IND
4,2014-10-11,103 ST-CORONA 7 IRT,13558.0,9128.0,2.0,103 ST-CORONA 7 IRT,103rd St 7 IRT


In [102]:
len(df_merged)

364104

In [103]:
subways.head(3)

,Division,Station_Name,Station_Latitude,Station_Longitude,LINENAME,uni
0,BMT,Ditmars Blvd,40.775036,-73.912034,NQ,Ditmars Blvd NQ BMT
1,BMT,Astoria Blvd-Hoyt Av,40.770258,-73.917843,NQ,Astoria Blvd-Hoyt Av NQ BMT
2,BMT,30 Av-Grand Av,40.766779,-73.921479,NQ,30 Av-Grand Av NQ BMT


In [104]:
df_merged = pd.merge(df_merged, subways, left_on='Map', right_on='uni')

In [105]:
len(df_merged)

377325

In [106]:
df_merged.columns

Index([u'DATE', u'uni_x', u'ENTRIES_DIFF', u'EXITS_DIFF', u'StationID',
       u'stations', u'Map', u'Division', u'Station_Name', u'Station_Latitude',
       u'Station_Longitude', u'LINENAME', u'uni_y'],
      dtype='object')

In [107]:
df_merged.head()

,DATE,uni_x,ENTRIES_DIFF,EXITS_DIFF,StationID,stations,Map,Division,Station_Name,Station_Latitude,Station_Longitude,LINENAME,uni_y
0,2014-10-11,1 AVE L BMT,13150.0,15662.0,75.0,1 AVE L BMT,1st Av L BMT,BMT,1st Av,40.730953,-73.981628,L,1st Av L BMT
1,2014-10-12,1 AVE L BMT,17117.0,20513.0,75.0,1 AVE L BMT,1st Av L BMT,BMT,1st Av,40.730953,-73.981628,L,1st Av L BMT
2,2014-10-13,1 AVE L BMT,12663.0,18642.0,75.0,1 AVE L BMT,1st Av L BMT,BMT,1st Av,40.730953,-73.981628,L,1st Av L BMT
3,2014-10-14,1 AVE L BMT,14588.0,21332.0,75.0,1 AVE L BMT,1st Av L BMT,BMT,1st Av,40.730953,-73.981628,L,1st Av L BMT
4,2014-10-15,1 AVE L BMT,14857.0,23543.0,75.0,1 AVE L BMT,1st Av L BMT,BMT,1st Av,40.730953,-73.981628,L,1st Av L BMT


In [108]:
df_merged.drop([u'uni_x',u'stations', u'Map',
                u'uni_y'], inplace = True, axis = 1)

In [112]:
df_merged.head()

,DATE,ENTRIES_DIFF,EXITS_DIFF,StationID,Division,Station_Name,Station_Latitude,Station_Longitude,LINENAME
0,2014-10-11,13150.0,15662.0,75.0,BMT,1st Av,40.730953,-73.981628,L
1,2014-10-12,17117.0,20513.0,75.0,BMT,1st Av,40.730953,-73.981628,L
2,2014-10-13,12663.0,18642.0,75.0,BMT,1st Av,40.730953,-73.981628,L
3,2014-10-14,14588.0,21332.0,75.0,BMT,1st Av,40.730953,-73.981628,L
4,2014-10-15,14857.0,23543.0,75.0,BMT,1st Av,40.730953,-73.981628,L


In [114]:
df_subways = df_merged.loc[:,['StationID','Division','Station_Name','Station_Latitude','Station_Longitude','LINENAME' ]]

In [116]:
df_subways.drop_duplicates(inplace = True)

In [119]:
df_subways.reset_index(inplace=True, drop=True)

In [120]:
df_subways.head()

,StationID,Division,Station_Name,Station_Latitude,Station_Longitude,LINENAME
0,75.0,BMT,1st Av,40.730953,-73.981628,L
1,76.0,BMT,1st Av,40.730953,-73.981628,L
2,77.0,BMT,1st Av,40.730953,-73.981628,L
3,0.0,IRT,103rd St,40.799446,-73.968379,1
4,1.0,IRT,103rd St,40.790600,-73.947478,6


In [121]:
df_subways.to_csv("../../Data/subway/subway_sta_id.csv")

In [122]:
df_merged.head()

,DATE,ENTRIES_DIFF,EXITS_DIFF,StationID,Division,Station_Name,Station_Latitude,Station_Longitude,LINENAME
0,2014-10-11,13150.0,15662.0,75.0,BMT,1st Av,40.730953,-73.981628,L
1,2014-10-12,17117.0,20513.0,75.0,BMT,1st Av,40.730953,-73.981628,L
2,2014-10-13,12663.0,18642.0,75.0,BMT,1st Av,40.730953,-73.981628,L
3,2014-10-14,14588.0,21332.0,75.0,BMT,1st Av,40.730953,-73.981628,L
4,2014-10-15,14857.0,23543.0,75.0,BMT,1st Av,40.730953,-73.981628,L


In [123]:
df_merged.to_csv("../../Data/subway/subway_station_counts.csv")

# Taxi Zone

In [69]:
tzones = gp.read_file("../Data/taxi_zones/taxi_zones_updated.shp")

In [75]:
tzones.to_crs(epsg=4326, inplace=True)

In [71]:
df_merged.head(2)

,DATE,ENTRIES_DIFF,EXITS_DIFF,Station_Name,geometry
0,2014-10-11,13150.0,15662.0,1st Av,POINT (-73.981628 40.730953)
1,2014-10-12,17117.0,20513.0,1st Av,POINT (-73.981628 40.730953)


In [76]:
tzones.head(3)

,LocationID,OBJECTID,Shape_Area,Shape_Leng,borough,geometry,zone
0,1,1,0.000782,0.116357,EWR,POLYGON ((-74.18445300000005 40.69499600003324...,Newark Airport
1,2,2,0.004866,0.433470,Queens,(POLYGON ((-73.82337597260654 40.6389870472100...,Jamaica Bay
2,3,3,0.000314,0.084341,Bronx,POLYGON ((-73.84792614099977 40.87134223403326...,Allerton/Pelham Gardens


In [81]:
df_merged['DATE'] = pd.to_datetime(df_merged["DATE"])

crs = {'init': 'epsg:4326', 'no_defs': True}
geometry = [Point(xy) for xy in zip(df_merged["Station_Longitude"], df_merged["Station_Latitude"])]
df_merged = df_merged.drop(["Station_Longitude", "Station_Latitude"], axis=1)
df_merged = gp.GeoDataFrame(df_merged, crs=crs, geometry=geometry)

print "Spatially joining data and taxi zones"

data_geo = gp.sjoin(tzones, df_merged) 
data_geo.drop(['borough', u'OBJECTID', 'Shape_Area', 'Shape_Leng',
                   u'geometry', 'index_right', 'zone', 'Station_Name'], axis = 1, inplace = True)   
    

data_agg = data_geo.set_index('DATE').groupby([pd.TimeGrouper('D'),'LocationID']).agg({'ENTRIES_DIFF':'sum', 
                                                                                      'EXITS_DIFF': 'sum'})
data_agg.reset_index('LocationID', inplace=True)

In [82]:
data_agg.head(3)

,LocationID,EXITS_DIFF,ENTRIES_DIFF
DATE,,,
2014-10-11,3,280.0,10.0
2014-10-11,7,27564.0,31367.0
2014-10-11,14,8609.0,16793.0


In [83]:
len(data_agg.LocationID.unique())

152

In [84]:
data_agg.to_csv("../Data/Aggregated/subways.csv")

In [85]:
data_agg.columns

Index([u'LocationID', u'EXITS_DIFF', u'ENTRIES_DIFF'], dtype='object')

In [87]:
data_agg.reset_index(inplace=True)

In [88]:
data_agg.DATE.describe()

count                  121999
unique                    812
top       2016-04-04 00:00:00
freq                      152
first     2014-10-11 00:00:00
last      2016-12-30 00:00:00
Name: DATE, dtype: object